In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = 10,
    mode = "gauge+number",
    title = {'text': "Performance"},
    # delta = {'reference': 380},
    gauge = {'axis': {'range': [-100, 50]},
             'bar': {'color': "white"},
             'steps' : [
                 {'range': [-100, 0], 'color': "#ea5b0c"},
                 {'range': [0, 25], 'color': "#f3d193"},
                 {'range': [25, 50], 'color': "#00ae9f"}],
             'threshold' : {'line': {'color': "white", 'width': 4}, 'thickness': 1, 'value': 10}}))

fig.update_layout(paper_bgcolor = "lavender", font = {'color': "grey", 'family': "Arial"})
fig.show()


In [3]:
def draw_guage(title: str, start:float, end:float, industry_average: float, value: float) -> go.Figure:

    bad_color = '#ea5b0c'
    good_color = '#00ae9f'
    middle_color = "#f3d193"
    
    bad_exists = True if start < 0 else False
    middle_exists = True if industry_average > 0 else False
    good_exists = True if end > 0 else False

    if not middle_exists:
        industry_average = 0
    if not good_exists:
        industry_average = end

    bad_region = {'range': [start, 0], 'color': bad_color} if bad_exists else None
    middle_region = {'range': [0, industry_average], 'color': middle_color} if middle_exists else None 
    good_region = {'range': [industry_average, end], 'color': good_color} if good_exists else None

    fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = value,
    mode = "gauge+number",
    title = {'text': title},
    # delta = {'reference': 380},
    gauge = {'axis': {'range': [start, end]},
             'bar': {'color': "white"},
             'steps' : [region for region in [bad_region, middle_region, good_region] if region != None],
             'threshold' : {'line': {'color': "white", 'width': 4}, 'thickness': 1, 'value': value}}))

    fig.update_layout(paper_bgcolor = "lavender", font = {'color': "grey", 'family': "Arial"})

    return fig



In [4]:
draw_guage("Performance",-100, 40, 10,-50)


In [5]:
def draw_guage_relative(title: str, start:float, end:float, industry_average: float, value: float) -> go.Figure:

    bad_color = '#ea5b0c'
    good_color = '#00ae9f'
    middle_color = "#f3d193"

    bad_range = [start, 0]
    middle_range = [0, industry_average]
    good_range = [industry_average, end]

    if end <= 0:
        good_range = [0, 0]

    if industry_average < 0:
        middle_range = [industry_average, 0]
        good_range = [0, end]
        middle_color = '#f69f71'
    

    bad_region = {'range': bad_range, 'color': bad_color}
    middle_region = {'range': middle_range, 'color': middle_color}
    good_region = {'range': good_range, 'color': good_color}


    fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = value,
    mode = "gauge+number",
    title = {'text': title},
    # delta = {'reference': 380},
    gauge = {'axis': {'range': [start, end]},
             'bar': {'color': "white"},
             'steps' : [region for region in [bad_region, middle_region, good_region] if region != None],
             'threshold' : {'line': {'color': "white", 'width': 4}, 'thickness': 1, 'value': value}}))

    fig.update_layout(paper_bgcolor = "lavender", font = {'color': "grey", 'family': "Arial"})

    return fig


In [6]:
draw_guage_relative('performance', -100, 50, -40, -20)

In [7]:
categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A',
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B',
      
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=True
)

fig.show()

In [8]:
spider_data = pd.DataFrame(
    {
        'categories': ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration'],
        'Industry Average': [1, 5, 2, 2, 3],
        'Your performance': [4, 3, 2.5, 1, 2]
    }
)
spider_data

,categories,Industry Average,Your performance
0,processing cost,1,4.0
1,mechanical properties,5,3.0
2,chemical stability,2,2.5
3,thermal stability,2,1.0
4,device integration,3,2.0


In [9]:
def draw_spider(df: pd.DataFrame, categories:str, values:list[str], fill=True):
    fig = go.Figure()
    min_value = None
    max_value = None

    
    for val in values:
        fig.add_trace(go.Scatterpolar(
            r=df[val],
            theta = df[categories],
            name=val,
            mode='lines',
            fill='toself' if fill else None,
        ))

        if min_value == None:
            min_value = df[val].min()
            max_value = df[val].max()
            continue

        new_min = df[val].min()
        new_max = df[val].max()
        if new_min < min_value:
            min_value = new_min
        if new_max > max_value:
            max_value = new_max


    fig.update_layout(
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[new_min*0.5, new_max*1.5]
        )),
    showlegend=True
    )

    return fig

draw_spider(spider_data, 'categories', ['Industry Average', 'Your performance'], True)

In [10]:
spider_data

,categories,Industry Average,Your performance
0,processing cost,1,4.0
1,mechanical properties,5,3.0
2,chemical stability,2,2.5
3,thermal stability,2,1.0
4,device integration,3,2.0


In [11]:
def draw_spider2(df: pd.DataFrame, categories:str, values:list[str], fill=True):
    melted_df = df.melt(id_vars=[categories], value_vars=values)
    # display(melted_df)
    fig = px.line_polar(
        melted_df, r='value', theta=categories, 
        color='variable', line_close=True,
    )
    if fill:
        fig.update_traces(fill='toself')
    return fig

draw_spider2(spider_data, 'categories', ['Industry Average', 'Your performance'])

In [12]:
ratio_data = pd.read_excel(r'ratio_df_combined.xlsx', sheet_name='ratios_clean')
ratio_data.head()

,Respondent,Current Ratio,Quick Ratio,FATO,TATO,Debt Ratio,Equity Multiplier,Gross Margin,Profit Margin,Return on Asset,Return on Equity,Withdrawal Ratio
0,4429,NaN,NaN,0.723207,0.519278,0.0,1.0,0.329987,0.196191,0.101878,0.101878,0.396820
1,4435,NaN,NaN,0.947232,0.164151,0.0,1.0,0.532442,-0.101485,-0.016659,-0.016659,-19.302141
2,4436,NaN,NaN,0.161418,0.161418,0.0,1.0,0.299856,-1.377132,-0.222294,-0.222294,-0.783240
3,4437,NaN,NaN,11.576842,0.786853,0.0,1.0,0.992908,0.539844,0.424777,0.424777,0.150239
4,4438,NaN,NaN,11.300565,2.167707,0.0,1.0,0.484097,0.263669,0.571556,0.571556,1.287214


In [13]:
demographic_data = pd.read_excel(r'lift_data\Expense Report - Mon Feb 13 2023.xlsx', sheet_name='Demographics')
demographic_data.head()

,Respondent ID,Gender,Age,Number of children,Marital Status,Country of Residence,Citizenship,Firm ID,Sector type,Date firm established,Age of Firm,Number of Owners,Owner/s Gender,Number of Employees,Industry,Location in the Country,Activation Date
0,4425,Male,35,3,Married,Ethiopia,Ethiopia,DD-ET-1930,AD,10/ 30/2016,7 - 10 years,1,All men,4,Other,Diredawa,2021-03-29
1,4429,Male,32,0,Single,Ethiopia,Ethiopia,AA-KG-SA-1376,EE-01,dd/03/2015,5 - 6 years,2,All men,18,Light Manu,Addis Ababa,2021-03-29
2,4431,Male,27,0,Single,Ethiopia,Ethiopia,AA-KG-SA-1495,Garment production,2018/03/22,2 years,4 or more,Both men and women,25,Light Manu,Addis Ababa,2021-03-29
3,4432,Male,32,2,Married,Ethiopia,Ethiopia,DD-ET-2194,U-03,0,5 - 6 years,1,All men,5,Other,Diredawa,2021-03-29
4,4433,Female,61,1,Widowed,Ethiopia,Ethiopia,HR-DE-2171,X,08/07/2017,3 - 4 years,3,Both men and women,12,Other,Harar,2021-03-29


In [14]:
demographic_data[['Respondent ID', 'Industry']].rename(columns={'Respondent ID': 'Respondent'})

,Respondent,Industry
0,4425,Other
1,4429,Light Manu
2,4431,Light Manu
3,4432,Other
4,4433,Other
...,...,...
154,4864,Light Manu
155,5045,Light Manu
156,5104,Other
157,5127,Other


In [15]:
ratio_demo_data = pd.merge(
    demographic_data[['Respondent ID', 'Industry']].rename(columns={'Respondent ID': 'Respondent'}),
    ratio_data,
    'outer', 
    'Respondent'
)
ratio_demo_data.head()

,Respondent,Industry,Current Ratio,Quick Ratio,FATO,TATO,Debt Ratio,Equity Multiplier,Gross Margin,Profit Margin,Return on Asset,Return on Equity,Withdrawal Ratio
0,4425,Other,44.843021,27.821745,0.305452,0.235211,0.005128,1.005155,0.823915,0.617714,0.145293,0.145293,1.191856
1,4429,Light Manu,NaN,NaN,0.723207,0.519278,0.000000,1.000000,0.329987,0.196191,0.101878,0.101878,0.396820
2,4431,Light Manu,NaN,NaN,0.006458,0.004768,0.000000,1.000000,-2.071429,-2.951429,-0.014074,-0.014074,-1.082285
3,4432,Other,NaN,NaN,0.012265,0.012190,0.000000,1.000000,1.000000,0.637209,0.007767,0.007767,0.211679
4,4433,Other,NaN,NaN,1.321050,0.493685,0.000000,1.000000,0.997352,0.516852,0.255162,0.255162,1.579040


In [16]:
ratio_demo_data.columns

Index(['Respondent', 'Industry', 'Current Ratio', 'Quick Ratio', 'FATO',
       'TATO', 'Debt Ratio', 'Equity Multiplier', 'Gross Margin',
       'Profit Margin', 'Return on Asset', 'Return on Equity',
       'Withdrawal Ratio'],
      dtype='object')

In [17]:
# ratio types
RES = 'Respondent'
INDUSTRY = 'Industry'
LIQUDITY = 'Liqudity'
PROFITABIITY = 'Profitability'
ASSET_MGMT = 'Asset management'
DEBT_MGMT = 'Debt management'
SAVINGS = 'Savings'
CR = 'Current Ratio'
PM = 'Profit Margin'
EM = 'Equity Multiplier'
WR = 'Withdrawal Ratio'
TATO = 'TATO'


def get_ratios_for(res_id:int, ratio_df:pd.DataFrame):
    filtered = ratio_df[ratio_df[RES] == res_id].reset_index().drop(columns='index')
    filtered.loc[:,SAVINGS] = filtered[WR].apply(lambda x: 1 - x if x >= 0 else x - 1)
    to_return = filtered.rename(columns={
        CR: LIQUDITY,
        PM: PROFITABIITY,
        EM: DEBT_MGMT,
        TATO: ASSET_MGMT
    })[[RES, INDUSTRY, LIQUDITY, PROFITABIITY, ASSET_MGMT, DEBT_MGMT, SAVINGS]]
    return to_return

def get_industry_data_for(res_id: int, ratio_df: pd.DataFrame):
    industry_name = ratio_df[ratio_df[RES] == res_id].reset_index().drop(columns='index')[INDUSTRY][0]
    # print(industry_name)
    industry_data = ratio_df[ratio_df[INDUSTRY] == industry_name]
    performances = []
    for res in industry_data[RES].unique():
        performances.append(get_ratios_for(res, ratio_df))
    
    return pd.concat(performances).reset_index().drop(columns='index')
    
display(get_ratios_for(4425, ratio_demo_data))
get_industry_data_for(4425, ratio_demo_data).head()

,Respondent,Industry,Liqudity,Profitability,Asset management,Debt management,Savings
0,4425,Other,44.843021,0.617714,0.235211,1.005155,-0.191856


,Respondent,Industry,Liqudity,Profitability,Asset management,Debt management,Savings
0,4425,Other,44.843021,0.617714,0.235211,1.005155,-0.191856
1,4432,Other,NaN,0.637209,0.012190,1.000000,0.788321
2,4433,Other,NaN,0.516852,0.493685,1.000000,-0.579040
3,4437,Other,NaN,0.539844,0.786853,1.000000,0.849761
4,4445,Other,NaN,0.325849,0.447482,1.000000,0.645613


In [18]:
def draw_speedometer_for(res_id:int, ratio_df:pd.DataFrame, ratio_type:str):
    industry_data = get_industry_data_for(res_id, ratio_df)
    industry_ratio = industry_data[[RES, ratio_type]]
    res_ratio = industry_ratio[industry_ratio[RES] == res_id].reset_index().drop(columns='index')[ratio_type][0]
    if pd.isna(res_ratio):
        return None
    start = industry_ratio[ratio_type].min()
    end = industry_ratio[ratio_type].max()
    value = res_ratio
    avg = industry_ratio[ratio_type].mean()

    return draw_guage_relative(ratio_type, start, end, avg, value)

draw_speedometer_for(4433, ratio_demo_data, SAVINGS)


In [19]:
def draw_spider_for(res_id: int, ratio_df: pd.DataFrame):
    industry_data = get_industry_data_for(res_id, ratio_df)
    res_ratios = industry_data[industry_data[RES] == res_id].reset_index().drop(columns='index')
    ratio_types = [LIQUDITY, PROFITABIITY, ASSET_MGMT, DEBT_MGMT, SAVINGS]
    firm_values = []
    industry_values = []
    for rt in ratio_types:
        min_value = industry_data[rt].min()
        max_value = industry_data[rt].max()
        r_value = res_ratios[rt][0]
        firm_percentage = (r_value - min_value) * 100/ (max_value - min_value)
        
        middle = industry_data[rt].mean()
        industry_percentage = (middle - min_value) * 100 / (max_value - min_value)

        firm_values.append(firm_percentage)
        industry_values.append(industry_percentage)
    
    spider_df = pd.DataFrame(
        {
            'Type': ratio_types,
            'Industry': industry_values,
            'You': firm_values
        }
    )
    
    spider_df2 = spider_df[~spider_df['You'].isna()]
    # display(spider_df2)
    return draw_spider2(spider_df2, 'Type', ['Industry', 'You'], False)

draw_spider_for(4618, ratio_demo_data)
